This is some interesting maths...

$\sum{}_{i=1}^{n} i = \frac{n(n+1)}{2}$

In [4]:
#imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from import_OMNI import read_OMNI, nan_value
from entropy import H, MI, CMI, entropy_matrix

%matplotlib inline

In [14]:
df = read_OMNI(
    filepath="../datasets/test_2.txt",
    formatpath="../data_format/test_2_format.txt",
)
print(list(df))


C:\Users\Aarnav\AppData\Local\Temp\ipykernel_12872\1038594709.py:2: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(


['5', 'Field', 'magnitude', 'average,', 'nT', 'F8.2']
['6', 'BX,', 'nT', '(GSE,', 'GSM)', 'F8.2']
['7', 'BY,', 'nT', '(GSE)', 'F8.2']
['8', 'BZ,', 'nT', '(GSE)', 'F8.2']
['9', 'BY,', 'nT', '(GSM)', 'F8.2']
['10', 'BZ,', 'nT', '(GSM)', 'F8.2']
['11', 'RMS', 'SD', 'B', 'scalar,', 'nT', 'F8.2']
['12', 'RMS', 'SD', 'field', 'vector,', 'nT', 'F8.2']
['13', 'Speed,', 'km/s', 'F8.1']
['14', 'Vx', 'Velocity,km/s', 'F8.1']
['15', 'Vy', 'Velocity,', 'km/s', 'F8.1']
['16', 'Vz', 'Velocity,', 'km/s', 'F8.1']
['17', 'Proton', 'Density,', 'n/cc', 'F7.2']
['18', 'Proton', 'Temperature,', 'K', 'F9.0']
['19', 'Na/Np', 'Ratio', 'F7.3']
['20', 'Flow', 'pressure,', 'nPa', 'F6.2']
['21', 'Electric', 'field,', 'mV/m', 'F7.2']
['22', 'Plasma', 'beta', 'F7.2']
['23', 'Alfven', 'mach', 'number', 'F6.1']
['24', 'Magnetosonic', 'Mach', 'number', 'F5.1']
['25', 'AE-index,', 'nT', 'I6']
['26', 'AL-index,', 'nT', 'I6']
['27', 'AU-index,', 'nT', 'I6']
['28', 'SYM/D,', 'nT', 'I6']
['29', 'SYM/H,', 'nT', 'I6']
['30', 

In [15]:
for column in list(df)[1:]:
    plt.plot(df["Datetime"], df[column], label=column)
# plt.plot(df["Datetime"], df["BZ"], label="BZ")
# plt.plot(df["Datetime"], df["SYM/H"], label="SYM/H")
# plt.hist(df["SYM/H"], 150)
plt.legend()
plt.show()

KeyboardInterrupt: 

In [16]:
entropy_matrix(df)

array([[ 5.55976548e+00,  4.65679295e-01,  7.65106012e-01,
         4.70867438e-01,  7.07731506e-01,  5.18581533e-01,
         3.04081145e-02,  4.38202071e-02,  1.77681147e-01,
         1.76053925e-01,  1.09143531e-01,  7.89954062e-02,
         2.11222491e-01,  7.10976631e-02, -1.40563291e-01,
         2.73751150e-01,  4.03975911e-01,  1.79541439e-01,
         5.94793143e-01,  4.78692711e-01,  7.25205333e-02,
        -3.22238835e-04, -4.89611645e-01,  7.98533207e-02,
         1.05168611e-01, -5.50571948e-01,  1.83327189e-01],
       [ 4.65679295e-01,  5.87061161e+00,  2.88259349e-01,
         9.07583605e-02,  2.67364701e-01,  9.06269219e-02,
         1.82433713e-02,  2.51199803e-02,  1.36384267e-01,
         1.34526508e-01,  9.22178070e-02,  5.25601837e-02,
         9.36075911e-02,  6.32671625e-02, -2.04705126e-01,
         1.20568413e-01,  9.05262286e-02,  4.74156413e-02,
         1.64597908e-01,  1.65660563e-01, -2.81707204e-02,
        -7.62001303e-02, -5.89313133e-01,  4.13977503e-